# Web scraping using BeautifulSoup and vkontakte.ru API

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import json

## Let's parse a page from a popular it website Habr. URL: habr.com/ru/all/

In [3]:
#keywords we will be looking for in parsed data
KEYWORDS = ['python', 'парсинг']

In [4]:
#getting the page itself
habr = requests.get('https://habr.com/ru/all/')
habr

<Response [200]>

In [5]:
#parsing the page using BeautifulSoup library
soup = BeautifulSoup(habr.text, 'html.parser')

In [6]:
#looking for all articles
posts = soup.find_all('div', class_='tm-article-snippet')
posts

[<div class="tm-article-snippet"><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" href="/ru/users/KaosEngineer/" title="KaosEngineer"><div class="tm-entity-image"><img alt="" class="tm-entity-image__pic" height="24" loading="lazy" src="//habrastorage.org/r/w32/getpro/habr/avatars/959/913/2f9/9599132f99c786ebb87b9026d10ae93c.jpg" width="24"/></div></a><span class="tm-user-info__user"><a class="tm-user-info__username" href="/ru/users/KaosEngineer/">
       KaosEngineer
     </a></span></span><span class="tm-article-snippet__datetime-published"><time datetime="2021-07-22T11:03:04.000Z" title="2021-07-22, 14:03">сегодня в 14:03</time></span></div><!-- --></div><h2 class="tm-article-snippet__title tm-article-snippet__title_h2"><a class="tm-article-snippet__title-link" data-article-link="" href="/ru/company/yandex/blog/568672/"><span>Яндекс открывает датасеты Беспилотны

In [45]:
#extracting dates for all articles
posts_date = list(map(lambda x: str(x.find('span', class_='tm-article-snippet__datetime-published').time)[16:40], posts))
posts_date

['2021-07-22T11:03:04.000Z',
 '2021-07-22T11:01:53.000Z',
 '2021-07-22T11:01:36.000Z',
 '2021-07-22T10:59:35.000Z',
 '2021-07-22T10:59:31.000Z',
 '2021-07-22T10:53:18.000Z',
 '2021-07-22T10:50:27.000Z',
 '2021-07-22T10:37:39.000Z',
 '2021-07-22T10:14:48.000Z',
 '2021-07-22T10:13:27.000Z',
 '2021-07-22T10:12:26.000Z',
 '2021-07-22T10:09:10.000Z',
 '2021-07-22T10:07:01.000Z',
 '2021-07-22T10:00:17.000Z',
 '2021-07-22T10:00:07.000Z',
 '2021-07-22T10:00:03.000Z',
 '2021-07-22T10:00:02.000Z',
 '2021-07-22T09:54:05.000Z',
 '2021-07-22T09:39:01.000Z',
 '2021-07-22T09:18:25.000Z']

In [8]:
#extracting headers for all articles
posts_title = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text, posts))
posts_title

['Яндекс открывает датасеты Беспилотных автомобилей, Погоды и Переводчика, чтобы помочь решить проблему сдвига данных в ML',
 'О небольших, но бесяще важных различиях текстовых редакторов',
 'Горячая перезагрузка .NET: новая возможность для редактирования кода во время выполнения приложений',
 'Radxa ROCK 3A: конкурент Raspberry Pi со слотом M.2 для NVMe SSD',
 'Памятка по жизненному циклу Android — часть I. Отдельные Activity',
 'Гайд: как создавать собственные активности для RPA-платформ',
 'Преобразуем CentOS в RHEL с помощью Convert2RHEL и Red Hat Satellite',
 'Как улучшить распознавание русской речи до 3% WER с помощью открытых данных',
 'История длиною в год: как мы на Greenplum 6 (DWH) мигрировали',
 'Тестирование или управление качеством. Часть 3. Что такое качество?',
 'Как мы SaaS решение переносили на сервера клиента. Стоит ли оно того?',
 'Мониторинг PostgreSQL. Расшифровка аудиочата Data Egret и Okmeter',
 'Многопользовательская сетевая игра Ticket to Ride',
 'Как и зачем 

In [17]:
#extracting links for all articles
posts_link = list(map(lambda x: 'https://habr.com' + x.find('a', class_='tm-article-snippet__title-link').get('href'), posts))
posts_link

['https://habr.com/ru/company/yandex/blog/568672/',
 'https://habr.com/ru/company/r7-office/blog/568942/',
 'https://habr.com/ru/company/otus/blog/569094/',
 'https://habr.com/ru/company/selectel/blog/568724/',
 'https://habr.com/ru/post/569092/',
 'https://habr.com/ru/company/uipath/blog/569090/',
 'https://habr.com/ru/company/redhatrussia/blog/569084/',
 'https://habr.com/ru/company/sberdevices/blog/569082/',
 'https://habr.com/ru/company/quadcode/blog/569056/',
 'https://habr.com/ru/company/otus/blog/569078/',
 'https://habr.com/ru/post/569076/',
 'https://habr.com/ru/company/flant/blog/568924/',
 'https://habr.com/ru/company/hsespb/blog/569070/',
 'https://habr.com/ru/company/arenadata/blog/566182/',
 'https://habr.com/ru/post/568798/',
 'https://habr.com/ru/post/568876/',
 'https://habr.com/ru/company/itsoft/blog/568998/',
 'https://habr.com/ru/company/factory5/blog/569066/',
 'https://habr.com/ru/post/569058/',
 'https://habr.com/ru/post/569054/']

In [38]:
#extracting text for all articles
posts_text = []
for link in posts_link:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    text = soup.find(id='post-content-body').text
    posts_text.append(text)
posts_text

['\n\r\nВ рамках конкурса Shifts Challenge мы выкладываем в открытый доступ крупнейший в мире датасет для обучения беспилотных автомобилей, а также данные Яндекс.Переводчика и Погоды. Приглашаем исследователей в области машинного обучения присоединиться к поиску решения проблемы сдвига распределения данных в реальном мире по отношению к тому, с чем моделям приходится иметь дело при обучении.\n\r\nМеня зовут Андрей Малинин, я старший исследователь в Yandex Research. Сегодня я расскажу о проблеме, о наших датасетах, а также о конкурсе, который мы проводим в рамках международной конференции NeurIPS 2021 совместно с учеными из Оксфордского и Кембриджского университетов.\n\nСдвиг распределения данных\r\nЧтобы разработать модель, обычно используются три выборки данных: обучающая, отладочная и проверочная (те данные, с которыми модели придется работать в реальном мире). Считается, что эти три выборки, с одной стороны, независимы друг от друга (данные, например, в отладочной и проверочной выбо

In [43]:
#creating a dataframe with the following columns date, title, link, text
#while checking if article's header or text contain the keywords defined earlier
habr_posts = pd.DataFrame()
for date, title, link, text in zip(posts_date, posts_title, posts_link, posts_text):
    if any(keyword.lower() in text.lower() for keyword in KEYWORDS) or any(keyword.lower() in title.lower() for keyword in KEYWORDS):
        row = {'date': date, 'title': title, 'link': link, 'text': text}
        habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])
habr_posts.reset_index().drop(columns='index')

,date,title,link,text
0,2021-07-22T10:07:01.000Z,Многопользовательская сетевая игра Ticket to Ride,https://habr.com/ru/company/hsespb/blog/569070/,"Привет, Хабр! Мы — Тимофей Василевский, Сергей..."
1,2021-07-22T10:00:03.000Z,Искусство отладки FPGA: как сократить срок тес...,https://habr.com/ru/post/568876/,"Примеры FPGA-проектов на базе Nvidia Jetson, X..."
2,2021-07-22T09:54:05.000Z,Платформы анализа данных: что они умеют и как ...,https://habr.com/ru/company/factory5/blog/569066/,Рынок ИТ- продуктов переполнен предложениями п...


## Let's check if a random email has been hacked using Avast Hack Check

In [54]:
EMAIL = ['xxxxx@x.ru', 'yyyyy@y.com']
#link to a hidden API which Avast uses to check email for hacks
hidden_api_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
#mandatory headers
HEADERS = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
}

In [55]:
#extracting information regarding hacks in json format
email_check = requests.post(hidden_api_url, json={'emailAddresses': EMAIL}, headers=HEADERS)
email_check.text

'{"breaches":{"88":{"breachId":88,"site":"ir.netease.com","recordsCount":256475863,"description":"In October 2015, the Chinese internet and gaming company NetEase suffered a data breach that leaked hundreds of millions of user credentials. The dump contained users\' email addresses along with their plain-text passwords.","publishDate":"2016-11-07T00:00:00Z","statistics":{"usernames":0,"passwords":255433278,"emails":256475863}},"16587":{"breachId":16587,"site":"dubsmash.com","recordsCount":161162574,"description":"In December 2018, Dubsmash\'s database was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2019-03-07T00:00:00Z","statistics":{"usernames":159256428,"passwords":161154927,"emails":161146252}},"17075":{"breachId":17075,"site":"toondo.com","recordsCount":6034161,"description":"In July 2019, the online comic creation site ToonDoo was alleg

In [69]:
#transforming json into python dictionary
email_check_dict = json.loads(email_check.text)
email_check_dict.keys()

dict_keys(['breaches', 'data', 'summary'])

In [119]:
email_check_pd = pd.DataFrame()
#creating dataframe with the following columns email, date, breach_source, and description
for breach_id, breach_source in zip(email_check_dict['breaches'], email_check_dict['data']):
    row = {
        'email': list(email_check_dict['data'][breach_source].keys())[0],
        'date': pd.to_datetime(email_check_dict['breaches'][breach_id]['publishDate'], format='%Y-%m-%d').strftime('%Y-%m-%d'),
        'breach_source': breach_source,
        'description': email_check_dict['breaches'][breach_id]['description']
    }
    email_check_pd = pd.concat([email_check_pd, pd.DataFrame([row])])
email_check_pd.reset_index().drop(columns='index')

,email,date,breach_source,description
0,yyyyy@y.com,2016-11-07,havenly.com,"In October 2015, the Chinese internet and gami..."
1,yyyyy@y.com,2019-03-07,adobe.com,"In December 2018, Dubsmash's database was alle..."
2,yyyyy@y.com,2019-11-14,ir.netease.com,"In July 2019, the online comic creation site T..."
3,yyyyy@y.com,2017-06-28,houdao.com,"In April 2011, gaming forum Houdao's user data..."
4,yyyyy@y.com,2016-10-21,linkedin.com,"In 2012, online professional networking platfo..."
5,yyyyy@y.com,2020-08-06,dubsmash.com,"In June 2020, the online interior design servi..."
6,xxxxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
7,yyyyy@y.com,2016-10-21,toondo.com,"In October of 2013, criminals penetrated Adobe..."


## Let's write a script that will get 50 last posts from a given group in Vkontakte.ru, a popular social network within Russian speaking countries

In [2]:
GROUP = 'netology'
#you would need a vk access token in order to work with website's API
with open('vk_access_token.txt', 'r') as token:
    TOKEN = token.read().strip()

In [3]:
#defining a method to get group's posts
GROUP_REQUEST = 'https://api.vk.com/method/wall.get'
VERSION = '5.58'
SLEEP = 0.33

In [25]:
#defining mandatory parameters
params = {
    'owner_id': -30159897,
    'domain': f'https://vk.com/{GROUP}',
    'access_token': TOKEN,
    'v': VERSION,
    'count': 50
}

In [26]:
#getting posts themselves
vk_posts = requests.get(GROUP_REQUEST, params)
vk_posts

<Response [200]>

In [28]:
vk_netology_posts = pd.DataFrame()
#creating a dataframe with the following columns date and text
for post in vk_posts.json()['response']['items']:
    row = {
        'date': pd.to_datetime(post['date'], unit='s'),
        'text': post['text']
    }
    vk_netology_posts = pd.concat([vk_netology_posts, pd.DataFrame([row])])
vk_netology_posts.reset_index().drop(columns='index')

,date,text
0,2021-03-17 13:25:03,Весна — лучшее время для перемен. Чтобы узнать...
1,2021-03-28 09:23:00,"Всё, что окружает нас в повседневной жизни так..."
2,2021-03-27 12:57:00,В этой афише мероприятий онлайн-эфир о развити...
3,2021-03-27 09:08:00,Разместить ссылку на свой сайт — это тоже свое...
4,2021-03-26 13:57:00,"Каждый день дизайнеры меняют IT и digital, фор..."
5,2021-03-26 07:44:00,"Точно решили, кем хотите стать в диджитале, но..."
6,2021-03-25 15:17:00,Мудборд — неотъемлемая часть подготовки любого...
7,2021-03-25 08:10:00,"Почему Java лучший выбор для новичка, какие на..."
8,2021-03-24 13:57:00,Что для вас важнее всего в работе? \nКому-то д...
9,2021-03-24 07:27:00,🚀 Запустили бесплатный курс «Основы Figma» \n ...
